In [4732]:
import pandas as pd
import numpy as np
import os 

In [4733]:
cwd = os.getcwd()

df = pd.read_excel(cwd+"/B2B Data Provider-sample1.xlsx", index_col=0)

#replace all Nan values with -
df = df.replace(np.nan, "-")

In [4734]:
print(df.columns)

Index(['Starting Price - Monthly', 'Starting Price - Annually', 'Price_min',
       'Price_max', 'Pricing Range', 'Pricing comments',
       'Additional Pricing Info', 'Website', 'Contract', 'Solutions',
       'Unlimited Data Plan', 'CRM Integrations', 'SEPI', 'SalesTeamType',
       'Free Trial', 'ContactEnrichment', 'IntentData', 'Other', 'location',
       'credits', 'AP1', 'AP1_credit', 'AP2', 'AP2_credit', 'AP3',
       'AP3_credit', 'AP4', 'AP4_credit', 'AP5', 'AP5_credit', 'MP1',
       'MP1_credit', 'MP2', 'MP2_credit', 'MP3', 'MP3_credit', 'MP4',
       'MP4_credit', 'MP5', 'MP5_credit', 'Comments'],
      dtype='object')


In [4735]:
#removing unnecessary columns
df_soft = df.drop(['Additional Pricing Info','Website','Unlimited Data Plan', 'Pricing Range', 'Pricing comments','Free Trial','Other'], axis=1)
df_soft = df_soft.sort_values(by='location', ascending=False)
#drop the pricing from data and this can be used to create a feature matrix
df_pro = df_soft.drop(['Starting Price - Monthly', 'Starting Price - Annually','Price_min','Price_max'], axis=1)
#get rid of last few numerical columns
df_pro = df_pro.iloc[:, :8]

In [4736]:
df_pro.head()

,Contract,Solutions,CRM Integrations,SEPI,SalesTeamType,ContactEnrichment,IntentData,location
Company,,,,,,,,
ZoomInfo,Annual,"email,workphone,mobile","Microsoft Dynamics,HubSpot,Salesforce,Pipedriv...","Outreach,Salesloft","SMB,MidMarket,Enterprise",Yes,Yes,USA/NorthAmerica
Uplead,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot,Insightly,Pipedrive,Zoho,Mi...","Outreach,Salesloft,Reply,Woodpecker,Lemlist,Close","SMB,MidMarket,Enterprise",Yes,Yes,USA/NorthAmerica
Lead411,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamic...",-,"SMB,MidMarket,Enterprise",Yes,Yes,USA/NorthAmerica
Apollo,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot","Salesloft,Outreach","SMB,MidMarket,Enterprise",Yes,No,USA/NorthAmerica
Seamless,Annual,"email,workphone,mobile","Salesforce,HubSpot,Microsoft Dynamics,Keap,Zoh...","Salesloft,Outreach","SMB,MidMarket",Yes,No,USA/NorthAmerica


In [4737]:
#convert all the contract to list values
df_pro['Contract'] = df_pro['Contract'].str.replace(" or ", ",").str.lower().str.split(',')
#convert all solutions values to list 
df_pro['Solutions'] = df_pro['Solutions'].str.lower().str.split(",")
#convert all the CRM values to list
df_pro['CRM Integrations'] = df_pro['CRM Integrations'].str.lower().str.split(",")
#convert all SEPI values to list
df_pro['SEPI'] = df_pro['SEPI'].str.lower().str.split(",")
#convert all location values to list
df_pro['location'] = df_pro['location'].str.lower().str.split(",")
#convert all location values to list
df_pro['SalesTeamType'] = df_pro['SalesTeamType'].str.lower().str.split(",")

In [4738]:
def get_software_data_matrix(dataframe):

    for i, col in enumerate(dataframe.columns):

        for index,row in dataframe.iterrows():
            for column in row[col]:
                if (dataframe.at[index,col] == "Yes") or (dataframe.at[index,col] == "No"):
                    pass
                else:
                    dataframe.at[index,column] =1

    #replace the Yes and no values to 1 and 0 respectively for ContactEnrichment and IntentData columns
    dataframe = dataframe.replace('Yes', 1.0)
    dataframe = dataframe.replace('No', 0)


    #now drop all the initial columns that is not a matrix
    dataframe =dataframe.drop(['Contract','Solutions','CRM Integrations','SEPI','location', '-', 'SalesTeamType'], axis=1)

    #lastly replace all the nan values with 0
    dataframe= dataframe.replace(np.nan, 0)

    #lower casing all the column names
    dataframe.columns = dataframe.columns.str.lower()
    dataframe.columns = dataframe.columns.str.replace(" ","")

    return dataframe

df_pro = get_software_data_matrix(df_pro)
df_pro

,contactenrichment,intentdata,annual,monthly,email,workphone,mobile,microsoftdynamics,hubspot,salesforce,...,salesloft,reply,woodpecker,lemlist,close,smb,midmarket,enterprise,usa/northamerica,international
Company,,,,,,,,,,,,,,,,,,,,,
ZoomInfo,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
Uplead,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
Lead411,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
Apollo,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
Seamless,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
RocketReach,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
Lusha,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
Crunchbase,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
Snov.io,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


In [4739]:
#check the shape
df_pro.shape

(10, 35)

In [4740]:
#treated dataframe 
df_pro.columns

Index(['contactenrichment', 'intentdata', 'annual', 'monthly', 'email',
       'workphone', 'mobile', 'microsoftdynamics', 'hubspot', 'salesforce',
       'pipedrive', 'sugarcrm', 'zoho', 'insightly', 'copper', 'nimble',
       'infusionsoft', 'capsule', 'nutshell', 'base', 'vtiger', 'keap',
       'freshsales', 'freshworks', 'outreach', 'salesloft', 'reply',
       'woodpecker', 'lemlist', 'close', 'smb', 'midmarket', 'enterprise',
       'usa/northamerica', 'international'],
      dtype='object')

**Now the idea is to cross multiply the values user values with and find the weighted values**

## USER data

In [4741]:
df_user = pd.read_csv("user_data.csv", index_col=0)
df_user

,data,contract,price,contacts,duration,Location,features,salesTeam,crm,SEPI_tools
0,"['email', 'workphone']",['Annual'],20000,1000-5000,['Annual'],[],[],[''],[],[]


In [4742]:
df_user.dtypes


data          object
contract      object
price          int64
contacts      object
duration      object
Location      object
features      object
salesTeam     object
crm           object
SEPI_tools    object
dtype: object

In [4743]:
for i, col in enumerate(df_user.columns):
    if df_user[col].dtype == "int64" or df_user[col].dtype == "float64":
        pass
    else:
        if col == "Location":
            if df_user['Location'].item().replace("'", "").strip("[]") == "Global":
                df_user['Location'] = [['usa/northamerica', 'international']]
            else:
                df_user[col] = df_user[col].item().replace("'", "").strip("[]").lower()
                df_user[col] = df_user[col].str.split(",")
        else:
            df_user[col] = df_user[col].item().replace("'", "").strip("[]").lower()
            df_user[col] = df_user[col].str.split(",")
        
        
    

In [4744]:
df_user

,data,contract,price,contacts,duration,Location,features,salesTeam,crm,SEPI_tools
0,"[email, workphone]",[annual],20000,[1000-5000],[annual],[],[],[],[],[]


In [4745]:
def get_user_data_matrix(dataframe):
    
    
    for i, col in enumerate(dataframe.columns):
        if col not in ['price','contacts']:
            # if dataframe[col][0][0] == '"i dont have any crm"' or dataframe[col][0][0] == '"i dont have a sales engagement platform"':
            #     pass
            # else:
                if col in ['crm', 'SEPI_tools']:
                    total_tools = len(dataframe['SEPI_tools'].item()) + len(dataframe['crm'].item())
                    for index,row in dataframe.iterrows():
                        for item in row[col]:
                            dataframe.at[index,item] = 1+(1/total_tools)
                elif col in ['Location']:
                    total_tools = len(dataframe['SEPI_tools'].item()) + len(dataframe['crm'].item())
                    for index,row in dataframe.iterrows():
                        for item in row[col]:
                            dataframe.at[index,item] = 1.5+(1/len(dataframe[col]))
                else:
                    for index,row in dataframe.iterrows():
                        for item in row[col]:
                            dataframe.at[index,item] =1+(1/ len(dataframe[col]))
            
    
    
    #now drop all the initial columns that is not a matrix
    dataframe = dataframe.iloc[:,10:]

    #lastly replace all the nan values with 0
    dataframe= dataframe.replace(np.nan, 0)

    #lower casing all the column names
    dataframe.columns = dataframe.columns.str.lower()
    
    #lower casing all the column names
    dataframe.columns = dataframe.columns.str.strip(" ")
   
    
    if "crm" in dataframe.columns:
        dataframe.drop(['crm'], axis=1, inplace=True)
    
    if 'SEPI_tools' in dataframe.columns:
        dataframe.drop(['SEPI_tools'], axis=1, inplace=True)
        
    return dataframe


df_user_matrix = get_user_data_matrix(df_user)

In [4746]:

df_user_matrix

,email,workphone,annual,
0,2.0,2.0,2.0,1.5


In [4747]:
df_pro.columns

Index(['contactenrichment', 'intentdata', 'annual', 'monthly', 'email',
       'workphone', 'mobile', 'microsoftdynamics', 'hubspot', 'salesforce',
       'pipedrive', 'sugarcrm', 'zoho', 'insightly', 'copper', 'nimble',
       'infusionsoft', 'capsule', 'nutshell', 'base', 'vtiger', 'keap',
       'freshsales', 'freshworks', 'outreach', 'salesloft', 'reply',
       'woodpecker', 'lemlist', 'close', 'smb', 'midmarket', 'enterprise',
       'usa/northamerica', 'international'],
      dtype='object')

In [4748]:
#match columns and drop the ones that do not match
for i,item in enumerate(df_pro):
    if item not in df_user_matrix.columns:
         df_pro.drop(item, axis=1, inplace=True)
    else:
        for index in range(len(df_pro)):
            df_pro[item][index] = df_pro[item][index] * df_user_matrix[item][0]
        
#match columns and drop the ones that do not match
for i,item in enumerate(df_user_matrix):
    if item not in df_pro.columns:
        df_user_matrix.drop(item, axis=1, inplace=True)

#making a dot product between the two matrices to get the probability score 
#df_pro = df_pro.dot(df_user_matrix.T)

In [4749]:
df_pro.shape, df_user_matrix.shape

((10, 3), (1, 3))

In [4750]:
df_pro

,annual,email,workphone
Company,,,
ZoomInfo,2.0,2.0,2.0
Uplead,2.0,2.0,2.0
Lead411,2.0,2.0,2.0
Apollo,2.0,2.0,2.0
Seamless,2.0,2.0,2.0
RocketReach,2.0,2.0,2.0
Lusha,2.0,2.0,2.0
Crunchbase,2.0,2.0,0.0
Snov.io,2.0,2.0,0.0


In [4751]:
df_pro.sum(axis=1)

Company
ZoomInfo       6.0
Uplead         6.0
Lead411        6.0
Apollo         6.0
Seamless       6.0
RocketReach    6.0
Lusha          6.0
Crunchbase     4.0
Snov.io        4.0
Cognism        6.0
dtype: float64

In [4752]:
df_soft['recommend']= df_pro.sum(axis=1)
#add back the pricing range column 
df_soft['Pricing Range'] = df['Pricing Range']


In [4753]:
def get_price_match(df_1, df_2):
    
    df_1['Starting Price - Monthly'] = df_1['Starting Price - Monthly'].str.replace(" ","").str.split(",")
    df_1['Starting Price - Annually'] = df_1['Starting Price - Annually'].str.replace(" ","").str.split(",")
    df_1['closet_price_diff'] = 0
    if "monthly" in df_2['duration'].item() and len(df_2['duration'].item())==1:
        
        for i in range(len(df_1)):
            diff = 0
            df_1['closet_price_diff'][i] = 50000
            
            for j in range(len(df_1['Starting Price - Monthly'][i])):
                diff =  abs(df_2['price'][0] - int(df_1['Starting Price - Monthly'][i][j]))
                if diff < df_1['closet_price_diff'][i]: 
                    df_1['closet_price_diff'][i] = diff
    elif "annual" in df_2['duration'].item() and len(df_2['duration'].item())==1 :
        
        for i in range(len(df_1)):
            diff = 0
            df_1['closet_price_diff'][i] = 50000
            for j in range(len(df_1['Starting Price - Annually'][i])):
                try:
                    diff =  abs(df_2['price'][0] - int(df_1['Starting Price - Annually'][i][j]))
                    if diff < df_1['closet_price_diff'][i]: 
                        df_1['closet_price_diff'][i] = diff
                except:
                    pass
    else:
        #df_1['closet_price_diff'] = df_1['Starting Price - Monthly']
        pass
        
    return df_1
            
            
df_soft = get_price_match(df_soft, df_user)

C:\Users\hardi\AppData\Local\Temp\ipykernel_3636\679871148.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['closet_price_diff'][i] = 50000
C:\Users\hardi\AppData\Local\Temp\ipykernel_3636\679871148.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['closet_price_diff'][i] = diff


In [4754]:
if type(df_user['price'].item()) == int:
    if len(df_user['Location'].item()[0]) != 0:
        result = df_soft.sort_values(by=['location','closet_price_diff'], ascending=[False,True])
        result = result.sort_values(by='recommend', ascending=False)
    else:
        result = df_soft.sort_values(by='closet_price_diff', ascending=True)
        result = result.sort_values(by='recommend', ascending=False)
else:
    
    result = df_soft.sort_values(by='recommend', ascending=False)

In [4755]:
result

,Starting Price - Monthly,Starting Price - Annually,Price_min,Price_max,Contract,Solutions,CRM Integrations,SEPI,SalesTeamType,ContactEnrichment,...,MP3,MP3_credit,MP4,MP4_credit,MP5,MP5_credit,Comments,recommend,Pricing Range,closet_price_diff
Company,,,,,,,,,,,,,,,,,,,,,
Cognism,"[0, 0]","[16500, ]",16500,27500,Annual,"email,workphone,mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamics",Salesloft,"SMB,MidMarket,Enterprise",Yes,...,-,-,-,-,-,-,-,6.0,"16,500-27,500+",3500
ZoomInfo,"[0, 0]","[11995, 15995, 24995, 34995]",11995,34995,Annual,"email,workphone,mobile","Microsoft Dynamics,HubSpot,Salesforce,Pipedriv...","Outreach,Salesloft","SMB,MidMarket,Enterprise",Yes,...,-,-,-,-,-,-,-,6.0,"Single user Pro Lite: $11,995 – 5,000 credits ...",4005
Apollo,"[49, 99]","[468, , 948, 6000]",0,6000,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot","Salesloft,Outreach","SMB,MidMarket,Enterprise",Yes,...,-,-,-,-,-,-,"*email only, doesn't include phone numbers\n**...",6.0,"$49 per month, month to month - $6000 per year+",14000
RocketReach,"[59, 119, 299]","[468, 1188, 1992, 2988, 4968]",468,6000,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot","Salesloft,Outreach","SMB,MidMarket",No,...,300.0,500.0,-,-,-,-,"*email only, doesn't include phone numbers\n**...",6.0,$80 per month to $6000 per year+,15032
Uplead,"[99, 199, 399]","[890, 1790, 3590]",888,4788,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot,Insightly,Pipedrive,Zoho,Mi...","Outreach,Salesloft,Reply,Woodpecker,Lemlist,Close","SMB,MidMarket,Enterprise",Yes,...,399.0,100.0,-,-,-,-,-,6.0,"$888 annual, $99 per month, month to month - ...",16410
Seamless,"[0, 147]","[1764, 2346]",1764,9528,Annual,"email,workphone,mobile","Salesforce,HubSpot,Microsoft Dynamics,Keap,Zoh...","Salesloft,Outreach","SMB,MidMarket",Yes,...,-,-,-,-,-,-,-,6.0,$1764 per year to $9528 per year+,17654
Lead411,"[99, 199]","[900, 1800]",900,1800,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamic...",-,"SMB,MidMarket,Enterprise",Yes,...,-,-,-,-,-,-,-,6.0,"$99 per month, month to month-$1800+ per year ...",18200
Lusha,"[39, 69]","[348, 612]",348,1140,Annual or Monthly,"email,workphone","Salesforce,HubSpot","Salesloft,Outreach",SMB,Yes,...,-,-,-,-,-,-,*multiple options to increase credit amount,6.0,"$39 per monmth to $1,140 per year+",19388
Snov.io,"[39, 99, 189, 369, 738]","[396, 996, 1896, 3696, 7380]",39,7380,Annual,email,"HubSpot,Salesforce,Pipedrive,Nutshell,Zoho,Fre...",-,"SMB,MidMarket",No,...,189.0,20000.0,369.0,50000.0,738.0,unlimited,*email only\n**could be considered unlimited g...,4.0,"$39 per month to & 7,380",12620


## Now we will perform credit (contacts) matching

In [4756]:
#replace unlimited values with 100000 and - with np.nan
result = result.reset_index()
result = result.replace("unlimited",100000)
result = result.replace("-",np.nan)
#create an empty column to store closest credits
result['credit_rank'] = 0

In [4757]:
def credit_fider(df_user_data,df_result, start_column, end_column):
    top_5 = []
    below_top_5 = []
    for i, col in enumerate(df_result.iloc[:,start_column:end_column].columns):
        if i%2 !=0:
         
            for index, item in enumerate(df_result[col]):
                if df_user_data['contacts'].item() == 100:
                    if item <= df_user_data['contacts'].item():
                        if index <=4:
                            top_5.append((index, item))
                            print(index, i,item , col)
                        else:
                            below_top_5.append((index, item))

                elif len(df_user_data['contacts'].item()[0].split("-")) ==2:

                    if (item >= int(df_user_data['contacts'].item()[0].split("-")[0]) and item <= int(df_user_data['contacts'].item()[0].split("-")[1])) :
                        if index <=4:
                            top_5.append((index, item))
                            print(index, i,item , col)
                        else:
                            below_top_5.append((index, item))

                elif df_user_data['contacts'].item()[0] == "more than 5000":
                    
                    if item > 5000:
                        if index <=4:
                            top_5.append((index, item))
                            print(index, i,item , col)
                        else:
                            below_top_5.append((index, item))
    
    return top_5, below_top_5
                        

In [4758]:

    
def get_credit_index(df_user_data, df_result):
    if "monthly" in df_user_data['duration'].item() and len(df_user_data['duration'].item())==1:
        #result = result.iloc[:,23:33]
        top_5, below_top_5 = credit_fider(df_user_data,df_result, 24, 34)
        return top_5, below_top_5

    elif "annual" in df_user_data['duration'].item() and len(df_user_data['duration'].item())==1 :
        #result = result.iloc[:,13:23]
        top_5, below_top_5 = credit_fider(df_user_data,df_result, 14, 24)
        return top_5, below_top_5
        
                            
    
                  

In [4759]:
try:
    #call the functions and get the value
    top_5_pro, below_top_5_pro = get_credit_index(df_user, result)
    
    #sort the list values
    top_5_pro.sort()
    
    if len(top_5_pro) != 0:
        for i in range(len(top_5_pro)):
            result['credit_rank'][top_5_pro[i][0]] = top_5_pro[i][1]
    else:
        pass
    
    result = result.sort_values(by='credit_rank', ascending=False).reset_index(drop=True).set_index('Company')
except:
    print("please fill in the values")
    


1 1 5000 AP1_credit
3 1 1200 AP1_credit
4 1 2040 AP1_credit
1 3 5000.0 AP2_credit
3 3 3600.0 AP2_credit
4 3 4800.0 AP2_credit


C:\Users\hardi\AppData\Local\Temp\ipykernel_3636\199087534.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['credit_rank'][top_5_pro[i][0]] = top_5_pro[i][1]


In [4760]:
result

,Starting Price - Monthly,Starting Price - Annually,Price_min,Price_max,Contract,Solutions,CRM Integrations,SEPI,SalesTeamType,ContactEnrichment,...,MP3_credit,MP4,MP4_credit,MP5,MP5_credit,Comments,recommend,Pricing Range,closet_price_diff,credit_rank
Company,,,,,,,,,,,,,,,,,,,,,
ZoomInfo,"[0, 0]","[11995, 15995, 24995, 34995]",11995,34995,Annual,"email,workphone,mobile","Microsoft Dynamics,HubSpot,Salesforce,Pipedriv...","Outreach,Salesloft","SMB,MidMarket,Enterprise",Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,"Single user Pro Lite: $11,995 – 5,000 credits ...",4005,5000
Uplead,"[99, 199, 399]","[890, 1790, 3590]",888,4788,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot,Insightly,Pipedrive,Zoho,Mi...","Outreach,Salesloft,Reply,Woodpecker,Lemlist,Close","SMB,MidMarket,Enterprise",Yes,...,100.0,NaN,NaN,NaN,NaN,NaN,6.0,"$888 annual, $99 per month, month to month - ...",16410,4800
RocketReach,"[59, 119, 299]","[468, 1188, 1992, 2988, 4968]",468,6000,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot","Salesloft,Outreach","SMB,MidMarket",No,...,500.0,NaN,NaN,NaN,NaN,"*email only, doesn't include phone numbers\n**...",6.0,$80 per month to $6000 per year+,15032,3600
Cognism,"[0, 0]","[16500, ]",16500,27500,Annual,"email,workphone,mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamics",Salesloft,"SMB,MidMarket,Enterprise",Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,"16,500-27,500+",3500,0
Apollo,"[49, 99]","[468, , 948, 6000]",0,6000,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot","Salesloft,Outreach","SMB,MidMarket,Enterprise",Yes,...,NaN,NaN,NaN,NaN,NaN,"*email only, doesn't include phone numbers\n**...",6.0,"$49 per month, month to month - $6000 per year+",14000,0
Seamless,"[0, 147]","[1764, 2346]",1764,9528,Annual,"email,workphone,mobile","Salesforce,HubSpot,Microsoft Dynamics,Keap,Zoh...","Salesloft,Outreach","SMB,MidMarket",Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,$1764 per year to $9528 per year+,17654,0
Lead411,"[99, 199]","[900, 1800]",900,1800,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamic...",NaN,"SMB,MidMarket,Enterprise",Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,"$99 per month, month to month-$1800+ per year ...",18200,0
Lusha,"[39, 69]","[348, 612]",348,1140,Annual or Monthly,"email,workphone","Salesforce,HubSpot","Salesloft,Outreach",SMB,Yes,...,NaN,NaN,NaN,NaN,NaN,*multiple options to increase credit amount,6.0,"$39 per monmth to $1,140 per year+",19388,0
Snov.io,"[39, 99, 189, 369, 738]","[396, 996, 1896, 3696, 7380]",39,7380,Annual,email,"HubSpot,Salesforce,Pipedrive,Nutshell,Zoho,Fre...",NaN,"SMB,MidMarket",No,...,20000.0,369.0,50000.0,738.0,100000.0,*email only\n**could be considered unlimited g...,4.0,"$39 per month to & 7,380",12620,0


In [4761]:
df_user

,data,contract,price,contacts,duration,Location,features,salesTeam,crm,SEPI_tools,email,workphone,annual,
0,"[email, workphone]",[annual],20000,[1000-5000],[annual],[],[],[],[],[],2.0,2.0,2.0,1.5
